In [ ]:
import os
import numpy as np
import glob
import laspy
from collections import defaultdict

tile_size = 50  # Size of each tile in meters

output_directory = r"E:\ALS_DATA\NLS_20p_EVO_TILES"
input_folder = r"E:\ALS_DATA\NLS_20p_EVO_large_tiles"

east_min = 395000.0 # 395500.0 
east_max = 407000.0 # 400500.0 
north_min = 6783000.0 # 6783500.0 
north_max = 6795000.0 # 6788500.0 

# def get_tile_key(x, y, tile_size, x_min, y_min):
#     tile_x = int((x - x_min) // tile_size)
#     tile_y = int((y - y_min) // tile_size)
#     return tile_x, tile_y

def process_laz_files_incremental(input_folder, output_folder, tile_size, east_min, east_max, north_min, north_max):

    laz_files = glob.glob(os.path.join(input_folder, '**/*.laz'), recursive=True)
    print(f'Found {len(laz_files)} .laz files.')

    for laz_file in laz_files:
        print(f'\nProcessing: {laz_file}')
        point_cloud = laspy.read(laz_file)
        points = point_cloud.points

        # filter if needed
        # x, y, z = point_cloud.x, point_cloud.y, point_cloud.z
        # mask = (x >= east_min) & (x <= east_max) & (y >= north_min) & (y <= north_max)

        # if not np.any(mask):
        #     continue

        # print(f'Points in AOI: {np.count_nonzero(mask)}')

        # filtered_points = point_cloud.points[mask]
        # filtered_x = x[mask]
        # filtered_y = y[mask]

        # tile_dict = defaultdict(list)
        # for i in range(len(points)):
        #     tile_x, tile_y = get_tile_key(point_cloud.x[i], point_cloud.y[i], tile_size, east_min, north_min)
        #     tile_dict[(tile_x, tile_y)].append(i)

        x = point_cloud.x
        y = point_cloud.y

        tile_x = ((x - east_min) // tile_size).astype(int)
        tile_y = ((y - north_min) // tile_size).astype(int)

        tile_keys = np.stack([tile_x, tile_y], axis=1)

        keys, inverse = np.unique(tile_keys, axis=0, return_inverse=True)

        tile_dict = {tuple(k): np.where(inverse == idx)[0] for idx, k in enumerate(keys)}

        for (tile_x, tile_y), indices in tile_dict.items():
            
            x_start = east_min + tile_x * tile_size
            y_start = north_min + tile_y * tile_size

            tile_filename = f'tile_x_{int(x_start)+25}_y_{int(y_start)+25}.laz' # file name correspond to the center of the tile (x, y)
            tile_path = os.path.join(output_folder, tile_filename)


            header = laspy.LasHeader(version="1.4", point_format=point_cloud.header.point_format)
            header.offsets = point_cloud.header.offsets
            header.scales = point_cloud.header.scales

            tile_pcloud = laspy.LasData(header)
            tile_pcloud.points = points[indices]

            if os.path.exists(tile_path):
                existing = laspy.read(tile_path)
    
                # tile_pcloud.x = np.concatenate((existing.x, tile_pcloud.x))
                # tile_pcloud.y = np.concatenate((existing.y, tile_pcloud.y))
                # tile_pcloud.z = np.concatenate((existing.z, tile_pcloud.z))
                # tile_pcloud.intensity = np.concatenate((existing.intensity, tile_pcloud.intensity))
                # tile_pcloud.return_number = np.concatenate((existing.return_number, tile_pcloud.return_number))
                # tile_pcloud.number_of_returns = np.concatenate((existing.number_of_returns, tile_pcloud.number_of_returns))
                # tile_pcloud.scanner_channel = np.concatenate((existing.scanner_channel, tile_pcloud.scanner_channel))
                # tile_pcloud.scan_direction_flag = np.concatenate((existing.scan_direction_flag, tile_pcloud.scan_direction_flag))
                # tile_pcloud.edge_of_flight_line = np.concatenate((existing.edge_of_flight_line, tile_pcloud.edge_of_flight_line))
                # tile_pcloud.classification = np.concatenate((existing.classification, tile_pcloud.classification))
                # tile_pcloud.user_data = np.concatenate((existing.user_data, tile_pcloud.user_data))
                # tile_pcloud.scan_angle = np.concatenate((existing.scan_angle, tile_pcloud.scan_angle))
                # tile_pcloud.point_source_id = np.concatenate((existing.point_source_id, tile_pcloud.point_source_id))
                # tile_pcloud.gps_time = np.concatenate((existing.gps_time, tile_pcloud.gps_time))
                # tile_pcloud.Amplitude = np.concatenate((existing.Amplitude, tile_pcloud.Amplitude))
                # tile_pcloud.Reflectance = np.concatenate((existing.Reflectance, tile_pcloud.Reflectance))
                # tile_pcloud.Deviation = np.concatenate((existing.Deviation, tile_pcloud.Deviation))

                merged = laspy.LasData(existing.header)
                merged.points = np.hstack([existing.points, tile_pcloud.points])
                merged.write(tile_path)

                # tile_pcloud.write(tile_path)
                print(f'Appended to: {tile_path}')

            else:
                tile_pcloud.write(tile_path)
                print(f'Created tile: {tile_path}')

    print('\nAll tiles saved!')

process_laz_files_incremental(input_folder, output_directory, tile_size, east_min, east_max, north_min, north_max)

Found 144 .laz files.

Processing: E:\ALS_DATA\NLS_20p_EVO_large_tiles\M4143G1_4.laz
Created tile: E:\ALS_DATA\NLS_20p_EVO_TILES\tile_x_399025_y_6786025.laz
Created tile: E:\ALS_DATA\NLS_20p_EVO_TILES\tile_x_399025_y_6786075.laz
Created tile: E:\ALS_DATA\NLS_20p_EVO_TILES\tile_x_399025_y_6786125.laz
Created tile: E:\ALS_DATA\NLS_20p_EVO_TILES\tile_x_399025_y_6786175.laz
Created tile: E:\ALS_DATA\NLS_20p_EVO_TILES\tile_x_399025_y_6786225.laz
Created tile: E:\ALS_DATA\NLS_20p_EVO_TILES\tile_x_399025_y_6786275.laz
Created tile: E:\ALS_DATA\NLS_20p_EVO_TILES\tile_x_399025_y_6786325.laz
Created tile: E:\ALS_DATA\NLS_20p_EVO_TILES\tile_x_399025_y_6786375.laz
Created tile: E:\ALS_DATA\NLS_20p_EVO_TILES\tile_x_399025_y_6786425.laz
Created tile: E:\ALS_DATA\NLS_20p_EVO_TILES\tile_x_399025_y_6786475.laz
Created tile: E:\ALS_DATA\NLS_20p_EVO_TILES\tile_x_399025_y_6786525.laz
Created tile: E:\ALS_DATA\NLS_20p_EVO_TILES\tile_x_399025_y_6786575.laz
Created tile: E:\ALS_DATA\NLS_20p_EVO_TILES\tile_x_

In [ ]:
import os
import numpy as np
import glob
import laspy
from collections import defaultdict

# FIND MAX X MIN X MAX Y MIN Y COORDINATES

# input_laz_directory = r'C:\Users\telukkari\Documents\ALS_DATA\EVO'
input_folder = r"E:\ALS_DATA\NLS_20p_EVO_large_tiles"

global_min_x = 100000000
global_max_x = -1
global_min_y = 100000000
global_max_y = -1

for i, file in enumerate(os.listdir(input_folder)):
    if i % 10 == 0:
        print(i)
    if file.lower().endswith(".laz"):
        file_path = os.path.join(input_folder, file)
        las = laspy.read(file_path)
        points = np.vstack((las.x, las.y, las.z)).T

        min_x = np.min(points[:,0])
        max_x = np.max(points[:,0])
        min_y = np.min(points[:,1])
        max_y = np.max(points[:,1])

        if min_x < global_min_x: global_min_x = min_x
        if max_x > global_max_x: global_max_x = max_x
        if min_y < global_min_y: global_min_y = min_y
        if max_y > global_max_y: global_max_y = max_y

print(f"Min x {global_min_x}")
print(f"Max x {global_max_x}")
print(f"Min y {global_min_y}")
print(f"Max x {global_max_y}")

    

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
Min x 395000.0
Max x 406999.99
Min y 6783000.0
Max x 6794999.99
